In [1]:
import cv2
import numpy as np
image_path = 'data/lena.png'
img=cv2.imread(image_path)

In [2]:
#對明亮度做直方圖均衡
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_hsv[...,-1] = cv2.equalizeHist(img_hsv[...,-1])
img = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)

#矩形邊框
img_rect= cv2.rectangle(img, (60,40), (420,510), (0,0,255), 3)

#水平翻轉
img_hol= img_rect[:,::-1,:]

#圖片縮放0.5倍
img_res = cv2.resize(img_hol, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)



In [3]:
while True:
    cv2.imshow('img_res', img_res)
    
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break


In [8]:
#求新的邊框座標

point1=[60,40]
point2=[420,510]

fx=0.5
fy=0.5

# 透過四則運算計算鏡像後位置
# 確保點的位置一樣是左上跟右下，所以交換鏡像後的 x 座標 (y 座標做水平鏡像後位置不變)

point1[0] = -point1[0]
point2[0] = -point2[0]


resize_col = int(img_hol.shape[1]*fx)
resize_row = int(img_hol.shape[0]*fy)

# 建構 scale matrix

M_scale = np.array([[fx, 0, 0],[0, fy, 0]], dtype=np.float32)
img_hol = cv2.warpAffine(img_hol, M_scale, (resize_col, resize_row))

# 把左上跟右下轉為矩陣型式
bbox = np.array((point1, point2), dtype=np.float32)
#print('M_scale.shape={}, bbox.shape={}'.format(M_scale.shape, bbox.shape))

# 做矩陣乘法可以使用 `np.dot`, 為了做矩陣乘法, M_scale 需要做轉置之後才能相乘
homo_coor_result = np.dot(M_scale.T, bbox)
homo_coor_result = homo_coor_result.astype('uint8')
#print(homo_coor_result)
scale_point1 = tuple(homo_coor_result[0])
scale_point2 = tuple(homo_coor_result[1])
print('origin point1={}, origin point2={}'.format(point1, point2))
print('resize point1={}, resize point2={}'.format(scale_point1, scale_point2))

origin point1=[-60, 40], origin point2=[-420, 510]
resize point1=(226, 20), resize point2=(46, 255)
